In [1]:
using DataFrames
using Dates
using DotEnv
using HTTP
using JSON
using Plots
using Printf

In [2]:
DotEnv.config();

In [3]:
timeseries_url = @sprintf(
    "https://api.stlouisfed.org/fred/series/observations?series_id=%s&api_key=%s&file_type=json",
    "IPG2211A2N",
    ENV["FRED_API_KEY"])
r = HTTP.get(timeseries_url)

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Content-Type: application/json; charset=UTF-8
Server: Apache
Last-Modified: Fri, 15 Mar 2019 13:27:05 GMT
Expires: Wed, 27 Mar 2019 04:15:56 GMT
x-rate-limit-limit: 120
x-rate-limit-remaining: 120
Date: Wed, 27 Mar 2019 04:15:56 GMT
Transfer-Encoding: chunked
Connection: keep-alive, Transfer-Encoding

{"realtime_start":"2019-03-26","realtime_end":"2019-03-26","observation_start":"1600-01-01","observation_end":"9999-12-31","units":"lin","output_type":1,"file_type":"json","order_by":"observation_date","sort_order":"asc","count":962,"offset":0,"limit":100000,"observations":[{"realtime_start":"2019-03-26","realtime_end":"2019-03-26","date":"1939-01-01","value":"3.3842"},{"realtime_start":"2019-03-26","realtime_end":"2019-03-26","date":"1939-02-01","value":"3.4100"},{"realtime_start":"2019-03-26","realtime_end":"2019-03-26","date":"1939-03-01","value":"3.4875"},{"realtime_start":"2019-03-26","realtime_end":"2019-03-26","date":"1939-04-01","value":

In [4]:
obs_vec = JSON.parse(IOBuffer(r.body))["observations"]

962-element Array{Any,1}:
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-01-01","value"=>"3.3842")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-02-01","value"=>"3.4100")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-03-01","value"=>"3.4875")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-04-01","value"=>"3.5133")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-05-01","value"=>"3.5133")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-06-01","value"=>"3.5650")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-07-01","value"=>"3.5650")  
 Dict{String,Any}("realtime_start"=>"2019-03-26","realtime_end"=>"2019-03-26","date"=>"1939-08-01","value"=>"3.6167")  
 Dict{String,A

In [5]:
obs_df = DataFrame(permutedims(hcat(collect.(values.(obs_vec))...), (2, 1)), Symbol.(collect(keys(obs_vec[1]))))

,realtime_start,realtime_end,date,value
,String,String,String,String
1,2019-03-26,2019-03-26,1939-01-01,3.3842
2,2019-03-26,2019-03-26,1939-02-01,3.4100
3,2019-03-26,2019-03-26,1939-03-01,3.4875
4,2019-03-26,2019-03-26,1939-04-01,3.5133
5,2019-03-26,2019-03-26,1939-05-01,3.5133
6,2019-03-26,2019-03-26,1939-06-01,3.5650
7,2019-03-26,2019-03-26,1939-07-01,3.5650
8,2019-03-26,2019-03-26,1939-08-01,3.6167
9,2019-03-26,2019-03-26,1939-09-01,3.7200


In [6]:
for x in [:realtime_start, :realtime_end, :date]
    obs_df[x] = Date.(obs_df[x] ,"y-m-d")
end
obs_df[:value] = parse.(Float64, obs_df[:value])

962-element Array{Float64,1}:
   3.3842
   3.41  
   3.4875
   3.5133
   3.5133
   3.565 
   3.565 
   3.6167
   3.72  
   3.72  
   3.7458
   3.7458
   3.7717
   ⋮     
 103.3912
  93.1969
  93.3137
 103.6679
 113.7848
 113.8726
 101.8622
  95.2701
 103.3201
 112.7514
 122.6572
 118.3609

In [ ]:
plot(obs_df[:date], obs_df[:value],
    size = [750, 400],
    title = "Industrial Production: Electric and gas utilities",
    ylabel = "Index 2012=100", 
    label = "")